<a target="_blank" href="https://colab.research.google.com/github/UpstageAI/cookbook/blob/main/cookbooks/upstage/Solar-Full-Stack LLM-101/05_3_OracleDB.ipynb">
<img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
</a>

# Try10: 전체 순서 변경하고 클린 코드로 수정

STEP 1. KB 구축
STEP 2. samples에서 질문과 정답 가져오기
STEP 3. 답변 생성
STEP 4. 답변 추출 및 평가

# 더 자세히
STEP 1. KB 구축
- step 1.1 ewha.pdf에서 pdf parsing
- step 1.2 pdf parsing한 것을 cleaning text
- step 1.3 추가 KB - 직접 추가하는 것도 가능하고, 다른 api도 가져올 수 있음.
- step 1.4 전체 KB를 <Embedding 모델>로 임베딩하고 retriever 준비 - 토큰 너무 많아지는 것을 방지하기 위해 split을 해서 retriever 준비
- step 1.5 프롬프팅과 함께 <QA 모델> 준비 - QA chain 사용

STEP 2. samples에서 질문과 정답 가져오기 - csv 파일에서 -> list로 변경
#
STEP 3. 답변 생성 <llm 모델>
#
STEP 4. 답변 추출
#
STEP 5. 평가

STEP 1. KB 구축
- step 1.1 ewha.pdf에서 pdf parsing

In [22]:
!pip3 install pdfplumber pymupdf pandas

In [23]:
import pandas as pd
import pdfplumber
import fitz # PyMuPDF
from langchain.schema import Document

def extract_text_or_table(pdf_path, table_pages):
    """
    pdf_path: PDF 파일 경로
    table_pages: 표가 있는 페이지의 인덱스를 리스트로 제공 (0부터 시작)
    """
    documents = []

    # PDF 열기
    with pdfplumber.open(pdf_path) as pdf:
        for i in range(len(pdf.pages)):
            if i in table_pages:
                # 표와 텍스트를 동시에 추출
                page = pdf.pages[i]
                
                # 표 추출
                tables = page.extract_table()
                table_content = f"표가 감지된 페이지 {i + 1}:\n{tables if tables else '표 없음'}"
                
                # 표 외의 텍스트 추출
                all_text = page.extract_text() or ""
                table_text = page.filter(lambda obj: obj['object_type'] == 'rect').extract_text() or ""
                remaining_text = all_text.replace(table_text, "").strip()

                content = f"{table_content}\n표 외 텍스트:\n{remaining_text if remaining_text else '표 외 텍스트 없음'}"
            else:
                # 표가 없는 페이지: PyMuPDF로 텍스트 추출
                with fitz.open(pdf_path) as doc:
                    text = doc[i].get_text("text")
                    content = f"텍스트 페이지 {i + 1}:\n{text.strip() if text else '텍스트 없음'}"
            
            documents.append(Document(page_content=content, metadata={"page": i + 1}))
    
    return documents

- step 1.2 pdf parsing한 것을 cleaning text

In [ ]:
import re

def clean_extracted_text(text):
    # 문장 중간의 줄바꿈 제거
    cleaned_text = re.sub(r'(?<=[a-z,])\n(?=[a-z])', ' ', text)
    # 문장 끝 줄바꿈 유지
    cleaned_text = re.sub(r'(?<=[.?!])\s*\n', '\n', cleaned_text)
    
    return cleaned_text

- step 1.3 추가 KB - 직접 추가하는 것도 가능하고, 다른 api도 가져올 수 있음.

In [25]:
def generate_rich_query(question):
    """
    가능한 QA pairs 전부 추가할 수 있음.
    input: 직접 만든 QA pairs. CSV 형태로 넣기.
    output: 딕셔너리 형태로 ?? 반환.
    """
    return f"다음 질문에 답하기 위한 추가 정보: {question} 관련된 배경 지식, 정의, 또는 세부 사항."

- step 1.4 전체 KB를 <Embedding 모델>로 임베딩하고 retriever 준비 - 토큰 너무 많아지는 것을 방지하기 위해 split을 해서 retriever 준비

In [26]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import FAISS

def create_vector_store(text, embedding_model):
    """
    <첫 번째 모델: Embedding 모델 사용>
    input: text, embedding model 받아서
    output: 임베딩 반환
    """
    splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
    texts = splitter.split_text(text)
    return FAISS.from_texts(texts, embedding_model)

- step 1.5 프롬프팅과 함께 <QA 모델> 준비 - QA chain 사용

In [27]:
from langchain.chains import RetrievalQA

def create_qa_chain(vector_store, model, prompt_template):
    """
    <두 번째 모델: QA 모델 사용>
    input: 임베딩한 거, QA 모델, 프롬프팅 넣어서
    output: 만든 QA 모델 반환
    """
    retriever = vector_store.as_retriever()
    return RetrievalQA.from_chain_type(llm=model, retriever=retriever, return_source_documents=False, chain_type_kwargs={"prompt": prompt_template})

STEP 2. samples에서 질문과 정답 가져오기 - csv 파일에서 -> list로 변경

In [28]:
import pandas as pd

def read_data(csv_path):
    """
    input: sample.csv
    output: prompts, answers
    """
    data = pd.read_csv(csv_path)
    return data['prompts'].tolist(), data['answers'].tolist()

STEP 3. 답변 생성 <llm 모델>

In [29]:
# main 코드

STEP 4. 답변 추출

In [56]:
import re

def extract_answer(text):
    """
    input: 생성한 답변 전체
    output: 생성한 답변 전체 중에서 다른 것 빼고 답변 부분만 추출
    """
    match = re.search(r"\([A-Z]\)", text)  # 괄호와 알파벳 모두 추출
    return match.group(0) if match else "N/A"

STEP 5. 평가

In [31]:
# main 코드

# MAIN 실행

In [ ]:
import os
from langchain.embeddings import OpenAIEmbeddings
from langchain.prompts import PromptTemplate
from langchain.chat_models import ChatOpenAI

api_key = "키"
data_path = "경로"

# OpenAI API Key 설정
os.environ["OPENAI_API_KEY"] = "키"

if __name__ == "__main__":
    PDF_PATH = os.path.join(data_path, 'ewha.pdf')
    CSV_PATH = os.path.join(data_path, 'samples2.csv')
    TABLE_PAGES = [29] + list(range(38, 53))

    # STEP 1. KB 구축
    # step 1.1 ewha.pdf에서 pdf parsing
    documents = extract_text_or_table(PDF_PATH, TABLE_PAGES)
    # step 1.2 pdf parsing한 것을 cleaning text
    # 각 페이지의 텍스트에 적용, 새로운 변수를 만들지 않고 이런 식으로 처리하기!!!
    for doc in documents:
        if isinstance(doc.page_content, str):
            cleaned_documents = clean_extracted_text(doc.page_content)

    # step 1.3 추가 KB - 직접 추가하는 것도 가능하고, 다른 api도 가져올 수 있음.

    # step 1.4 전체 KB를 <Embedding 모델>로 임베딩하고 retriever 준비 - 토큰 너무 많아지는 것을 방지하기 위해 split을 해서 retriever 준비
    embeddings = OpenAIEmbeddings()
    vector_store = create_vector_store(cleaned_documents, embeddings)

    # step 1.5 프롬프팅과 함께 <QA 모델> 준비 - QA chain 사용
    llm = ChatOpenAI(model="gpt-3.5-turbo", temperature=0)
    custom_prompt = PromptTemplate(
        input_variables=["context", "question"],
        template="""다음 문서를 바탕으로 질문에 답하세요:\n{context}\n질문: {question}\n답변: """,
    )
    qa_chain = create_qa_chain(vector_store, llm, custom_prompt)

    # STEP 2. samples에서 질문과 정답 가져오기 - csv 파일에서 -> list로 변경
    prompts, answers = read_data(CSV_PATH)

    # STEP 4. 답변 추출
    correct = 0
    for i, question in enumerate(prompts):
        result = qa_chain.run(question)
        predicted_answer = extract_answer(result)
        print(f"질문 {i + 1}: {question}")
        print(f"생성된 답변: {result}")
        print(f"예상 답: {predicted_answer}, 실제 답: {answers[i]}\n")
        if predicted_answer == answers[i]:
            correct += 1

    # STEP 5. 평가
    accuracy = correct / len(prompts) * 100
    print(f"acc: {accuracy:.2f}%")


질문 1: QUESTION1) 영어 및 정보 등에 관하여 일정한 기준의 능력이나 자격을 취득한 경우 인정 받는 학점은 몇점인가?
(A) 인정 안됨
(B) 1학점
(C) 2학점
(D) 3학점
생성된 답변: (C) 2학점
예상 답: (C), 실제 답: (A)

질문 2: QUESTION2) LMS 시스템의 초기 비밀번호는 무엇으로 설정되어 있습니까?
(A) 사용자가 직접 설정
(B) 본인 생년월일
(C) 0000
(D) 임의 지정된 숫자
생성된 답변: (B) 본인 생년월일
예상 답: (B), 실제 답: (B)

질문 3: QUESTION3) 비대면 수업을 위해 필요한 필수 장비는 무엇입니까?
(A) 컴퓨터, 카메라, 마이크
(B) 필기구와 종이
(C) USB 드라이브
(D) 추가 모니터
생성된 답변: (A) 컴퓨터, 카메라, 마이크
예상 답: (A), 실제 답: (A)

질문 4: QUESTION4) 학생 포털에서 본인의 성적을 조회할 수 있는 시기는 언제입니까?
(A) 시험 직후
(B) 성적 입력 마감 후
(C) 학기 초
(D) 졸업 후에만 조회 가능
생성된 답변: (B) 성적 입력 마감 후
예상 답: (B), 실제 답: (B)

질문 5: QUESTION5) 온라인 수업 출석 인정 기준은 무엇입니까?
(A) 접속 후 5분 내 퇴장해도 인정됨
(B) 최소 80% 수업 시청
(C) 질문 게시판에 참여
(D) 시험 응시 여부로만 결정
생성된 답변: (B) 최소 80% 수업 시청
예상 답: (B), 실제 답: (B)

질문 6: QUESTION6) In Singer’s understanding, the principle of the equality of human beings is
(A) a description of an alleged actual equality among humans.
(B) a description of an alleged equality among all living beings.
(C) a prescription of how we shoul

In [47]:
# 결과 출력
for doc in documents:
    print(f"Page {doc.metadata['page']}:")
    print(cleaned_documents)

Page 1:
텍스트 페이지 54:
이화여자대학교 학칙
2 - 2 - 54
(서식 제1호)
(개정  2009.2.23)
     
졸  업  증  서
   성 명
        년    월    일생
     위 사람은 아래의 과정을 이수하고 소정의 자격을 취득하였기에 이 증서
를 수여함.
전  공    ○○○○ (○○학사)
          ○○○○ (○○학사)
          ○○○○ (○○학사)
부전공    ○○○○
                             년     월     일
이화여자대학교 총   장    ○   ○   ○    (인)
   학위번호:  이화여대 ○○○○ (학) ○○○                 제    호
Page 2:
텍스트 페이지 54:
이화여자대학교 학칙
2 - 2 - 54
(서식 제1호)
(개정  2009.2.23)
     
졸  업  증  서
   성 명
        년    월    일생
     위 사람은 아래의 과정을 이수하고 소정의 자격을 취득하였기에 이 증서
를 수여함.
전  공    ○○○○ (○○학사)
          ○○○○ (○○학사)
          ○○○○ (○○학사)
부전공    ○○○○
                             년     월     일
이화여자대학교 총   장    ○   ○   ○    (인)
   학위번호:  이화여대 ○○○○ (학) ○○○                 제    호
Page 3:
텍스트 페이지 54:
이화여자대학교 학칙
2 - 2 - 54
(서식 제1호)
(개정  2009.2.23)
     
졸  업  증  서
   성 명
        년    월    일생
     위 사람은 아래의 과정을 이수하고 소정의 자격을 취득하였기에 이 증서
를 수여함.
전  공    ○○○○ (○○학사)
          ○○○○ (○○학사)
          ○○○○ (○○학사)
부전공    ○○○○
                           

In [48]:
print(cleaned_documents)

텍스트 페이지 54:
이화여자대학교 학칙
2 - 2 - 54
(서식 제1호)
(개정  2009.2.23)
     
졸  업  증  서
   성 명
        년    월    일생
     위 사람은 아래의 과정을 이수하고 소정의 자격을 취득하였기에 이 증서
를 수여함.
전  공    ○○○○ (○○학사)
          ○○○○ (○○학사)
          ○○○○ (○○학사)
부전공    ○○○○
                             년     월     일
이화여자대학교 총   장    ○   ○   ○    (인)
   학위번호:  이화여대 ○○○○ (학) ○○○                 제    호


In [50]:
print(result)

(B) 2027년까지 재학할 수 있다.
